In [17]:
import pandas as pd
import os

# Caminhos de entrada e saída
caminho_csv = 'C:\\AFL_GABRIEL\\csv_parser\\DATA\\sis\\2299x1010'
caminho_parquet = 'C:\\AFL_GABRIEL\\csv_parser\\DATA_PARQUET\\sis\\2299x1010'

# Nome do arquivo final Parquet
nome_arquivo_parquet = '2299x1010.parquet'
caminho_arquivo_parquet = os.path.join(caminho_parquet, nome_arquivo_parquet)

# Criar o diretório de saída, se necessário
os.makedirs(caminho_parquet, exist_ok=True)

# Lista para armazenar os DataFrames
lista_dfs = []

# Iterar sobre os arquivos CSV
for arquivo in os.listdir(caminho_csv):
    if arquivo.endswith('.csv'):
        caminho_arquivo = os.path.join(caminho_csv, arquivo)

        try:
            df = pd.read_csv(caminho_arquivo, delimiter="|", dtype=str)
            if not df.empty:
                lista_dfs.append(df)
                print(f'Lido: {arquivo} com {df.shape[0]} linhas')
            else:
                print(f'Ignorado (vazio): {arquivo}')
        except Exception as e:
            print(f'Erro ao ler {arquivo}: {e}')

# Verifica se há dados para salvar
if lista_dfs:
    df_final = pd.concat(lista_dfs, ignore_index=True)
    try:
        df_final.to_parquet(caminho_arquivo_parquet, index=False)
        print(f'Dados salvos em: {caminho_arquivo_parquet} ({df_final.shape[0]} linhas)')
    except Exception as e:
        print(f'Erro ao salvar o arquivo Parquet: {e}')
else:
    print('Nenhum dado válido encontrado. Nenhum arquivo Parquet foi criado.')

print('Conversão concluída!')


Lido: 2299_x_1010.csv com 37568 linhas
Dados salvos em: C:\AFL_GABRIEL\csv_parser\DATA_PARQUET\sis\2299x1010\2299x1010.parquet (37568 linhas)
Conversão concluída!


In [22]:
import pandas as pd
import pyperclip
import os

# Caminho da pasta com arquivos Parquet
caminho_parquet = 'C:\\AFL_GABRIEL\\csv_parser\\DATA_PARQUET\\sis\\1200x2200'

# Lista de palavras-chave para conversão e substituição
palavras_chave = ["Vlr", "Valor", "vlr", "valor"]  # Modifique conforme necessário

# Lista para armazenar DataFrames
lista_df = []

# Percorrer a pasta e juntar todos os arquivos .parquet
for arquivo in os.listdir(caminho_parquet):
    if arquivo.endswith('.parquet'):
        caminho_arquivo = os.path.join(caminho_parquet, arquivo)
        try:
            df_temp = pd.read_parquet(caminho_arquivo)
            lista_df.append(df_temp)
            print(f'Lido: {arquivo} ({df_temp.shape[0]} linhas)')
        except Exception as e:
            print(f'Erro ao ler {arquivo}: {e}')

# Junta todos os DataFrames lidos
df = pd.concat(lista_df, ignore_index=True)

# Informações do DataFrame
num_linhas, num_colunas = df.shape  # Obter número de linhas e colunas

# Criar a string com os resultados
resultado = f"🔹 Total de Linhas: {num_linhas}\n🔹 Total de Colunas: {num_colunas}\n\n📊 Detalhes das Colunas:\n"

# Percorre todas as colunas e adiciona detalhes
for coluna in df.columns:
    linhas_preenchidas = df[coluna].count()  # Conta valores não nulos
    
    # Verifica se a coluna está na lista de palavras-chave
    if any(palavra.lower() in coluna.lower() for palavra in palavras_chave):
            # Converte para float, ignorando erros caso tenha valores inválidos
            df[coluna] = (
                df[coluna].astype(str)
                .str.replace('.', '', regex=False)      # Remove milhar
                .str.replace(',', '.', regex=False)     # Converte decimal
                .str.extract(r'(\d+\.?\d*)')[0]          # Extrai número
                .astype(float)
            )
            # Soma
            soma_valores = df[coluna].sum(skipna=True)

            # Exibição em formato brasileiro
            valor_formatado = f"{soma_valores:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")

            resultado += f"\n📌 Coluna: {coluna} (🔄 Convertida para Float e Substituição de '.' por ',')\n"
            resultado += f"   🔹 Linhas Preenchidas: {linhas_preenchidas}\n"
            resultado += f"   🔹 Total Somado: {soma_valores:,.2f}\n"
    else:
        resultado += f"\n📌 Coluna: {coluna}\n"
        resultado += f"   🔹 Linhas Preenchidas: {linhas_preenchidas}\n"

# Copiar para a área de transferência
pyperclip.copy(resultado)

# Exibir os resultados
print("\n✅ Resultado copiado para a área de transferência!")


Lido: 1200x2200.parquet (1320256 linhas)

✅ Resultado copiado para a área de transferência!
